# DEMO solution
This notebook serves as first step in using NLP and machine learning to predict stock movements
This first iteration will focus on scraping headlines using newsapi.org API and create a dataset for sentiment analysis

## Getting data
Using newsapi.org to get headlines

In [16]:
# Getting API-key

import sys    

sys.path.append("C:/Users/peter/Documents/NLP-stock-project")

from keys import APIkey

# some more imports

import pprint
import requests 

In [17]:
# define endpoint
url = 'https://newsapi.org/v2/everything?'

# parameters
parameters = {
    'q': 'microsoft', # query phrase
    'from': '2023-10-08',  # Start date (YYYY-MM-DD) - specify your desired date here
    'to': '2023-10-10',    # End date (YYYY-MM-DD) - specify your desired end date
    'sortBy': 'publishedAt', # sort by publishingdate # number of articles (100 max)
    'apiKey': APIkey # your own API key
}

In [18]:
# try:
#     response = requests.get(url, params=parameters)
#     response.raise_for_status()  # Raise HTTPError for bad requests
#     response_json = response.json()
#     print(response_json)
# except requests.exceptions.HTTPError as errh:
#     print("HTTP Error:", errh)
# except requests.exceptions.ConnectionError as errc:
#     print("Error Connecting:", errc)
# except requests.exceptions.Timeout as errt:
#     print("Timeout Error:", errt)
# except requests.exceptions.RequestException as err:
#     print("OOps: Something Else", err)

In [19]:
# Make the request
response = requests.get(url, params=parameters)

# Convert the response to JSON format
response_json = response.json()

# Check out the dictionaries keys
#print(response_json.keys())

# get articles
articles = response_json["articles"]

In [20]:
len(articles)

100

In [12]:
# Get the total number of results
total_results = response_json["totalResults"]

# Print the total number of results
print(f"Total number of results: {total_results}")

Total number of results: 905


In [21]:
titles = list(map(lambda x: x["title"], articles))

In [7]:
pprint.pprint(response_json['articles'])

[{'author': None,
  'content': '(Updates with action in US tech stocks, updates first\r\n'
             'paragraph, adds quotes in paragraphs 8 and 9)\r\n'
             'SAN FRANCISCO/JERUSALEM Oct 9 (Reuters) - Tech companies\r\n'
             'operating in Israel are expected to for… [+4787 chars]',
  'description': '(marketscreener.com) Tech companies\n'
                 'operating in Israel are expected to fortify security as '
                 'they\n'
                 'could face disruptions, said investors and analysts, as the\n'
                 'Israeli military shifted to a war footing that may include '
                 'a\n'
                 'full-scale invasion of the Gaza Strip.\n'
                 ' H…',
  'publishedAt': '2023-10-09T19:34:44Z',
  'source': {'id': None, 'name': 'Marketscreener.com'},
  'title': "Israel's tech sector could face disruptions after attacks - "
           'investors',
  'url': 'https://www.marketscreener.com/quote/currency/EURO-ISRAELI-NEW-SHE

#### pagination loop

In [39]:
# importing datetime
from datetime import datetime, timedelta, date
import time

# bool
flag = True

# api endpoint
url = 'https://newsapi.org/v2/everything?'

# query params
query = 'microsoft'
start_date = date.today() -timedelta(days=14)
end_date = date.today()
sortBy = 'publishedAt'
api = APIkey
pagesize = 100

# empty list for titles
ms_titles = []

# creating date var for loop
current_date = start_date

while current_date < end_date:
    # to not crash API
    time.sleep(1)
    # api params
    parameters = {
        'q': query, # query phrase
        'from': current_date,  # Start date (YYYY-MM-DD) - specify your desired date here
        'to': current_date,    # End date (YYYY-MM-DD) - specify your desired end date
        'sortBy': sortBy, # sort by publishingdate # number of articles (100 max)
        'pageSize': pagesize,
        'apiKey': api, # your own API key
    }
    
    # calling the api and getting json
    response = requests.get(url, params=parameters)
    response_json = response.json()
    # get articles
    articles = response_json["articles"]
    
    # getting titles
    titles_per_page = map(lambda x: x["title"], articles)
    # extending list
    ms_titles.extend(titles_per_page)
    
    # update while variable
    current_date = current_date + timedelta(days=1)



In [40]:
len(ms_titles)

1400